 Install Libraries

In [1]:
# Uninstall conflicting packages first
!pip uninstall -y langchain langchain-community langchain-core langchain-groq -q

# Install compatible versions
!pip install -q langchain-core==0.3.29
!pip install -q langchain==0.3.13
!pip install -q langchain-community==0.3.13
!pip install -q langchain-groq==0.2.1
!pip install -q langchain-text-splitters==0.3.4
!pip install -q tiktoken chromadb langgraph tavily-python fastembed

print("Installation complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.29 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 40.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.63 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━

 Import Libraries

In [2]:
import os
from typing import List
from typing_extensions import TypedDict

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.documents import Document
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langgraph.graph import END, StateGraph

print("All imports successful!")

All imports successful!


Configure API Keys

In [ ]:
GROQ_API_KEY = ""
TAVILY_API_KEY = ""

os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

Initialize Models

In [4]:
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key=GROQ_API_KEY
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

load documents

In [5]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(f"Loaded {len(docs_list)} documents")

Loaded 3 documents


: Split Documents

In [6]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512,
    chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)
print(f"Created {len(doc_splits)} chunks")

Created 87 chunks


Create Vector Store | Embeddings

In [7]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embed_model,
    collection_name="local-rag"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
print("Vector store created")

Vector store created


Define State

In [8]:
class GraphState(TypedDict):
    question: str
    generation: str
    web_search: str
    documents: List[str]

 Create Router

In [9]:
router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert at routing a user question to a vectorstore or web search.
Use the vectorstore for questions on LLM agents, prompt engineering, and adversarial attacks.
Otherwise, use web-search.
Give a binary choice 'web_search' or 'vectorstore' based on the question.
Return a JSON with a single key 'datasource' and no preamble or explanation.

Question to route: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = router_prompt | llm | JsonOutputParser()

Create Retrieval Grader

In [10]:
retrieval_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keywords related to the user question, grade it as relevant.
Give a binary score 'yes' or 'no' to indicate whether the document is relevant.
Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Here is the retrieved document:
{document}

Here is the user question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

retrieval_grader = retrieval_grader_prompt | llm | JsonOutputParser()

create rag chan

In [11]:
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: {question}
Context: {context}

Answer:
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = rag_prompt | llm | StrOutputParser()

Create Hallucination Grader

In [12]:
hallucination_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a grader assessing whether an answer is grounded in / supported by a set of facts.
Give a binary 'yes' or 'no' score to indicate whether the answer is grounded in the facts.
Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Here are the facts:
{documents}

Here is the answer: {generation}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = hallucination_grader_prompt | llm | JsonOutputParser()

Create Answer Grader

In [13]:
answer_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a grader assessing whether an answer is useful to resolve a question.
Give a binary score 'yes' or 'no' to indicate whether the answer is useful.
Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Here is the answer:
{generation}

Here is the question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = answer_grader_prompt | llm | JsonOutputParser()

Initialize Web Search

In [14]:
web_search_tool = TavilySearchResults(k=3)

Define Node Functions

In [15]:
def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": format_docs(documents), "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE---")
    question = state["question"]
    documents = state["documents"]
    filtered_docs = []
    web_search = "No"
    
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        
        if grade.lower() == "yes":
            print("---GRADE: RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: NOT RELEVANT---")
            web_search = "Yes"
    
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def web_search(state):
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])
    
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    
    return {"documents": documents, "question": question}

: Define Edge Functions

In [16]:
def route_question(state):
    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    
    if source['datasource'] == 'web_search':
        print("---ROUTE: WEB SEARCH---")
        return "websearch"
    else:
        print("---ROUTE: VECTORSTORE---")
        return "vectorstore"

def decide_to_generate(state):
    print("---ASSESS DOCUMENTS---")
    web_search = state["web_search"]
    
    if web_search == "Yes":
        print("---DECISION: WEB SEARCH---")
        return "websearch"
    else:
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    score = hallucination_grader.invoke({"documents": format_docs(documents), "generation": generation})
    grade = score['score']
    
    if grade == "yes":
        print("---GROUNDED IN DOCUMENTS---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score['score']
        
        if grade == "yes":
            print("---ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---NOT GROUNDED---")
        return "not supported"

Build Workflow

In [17]:
workflow = StateGraph(GraphState)

workflow.add_node("websearch", web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)

workflow.add_edge("websearch", "generate")

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

app = workflow.compile()
print("Workflow compiled successfully!")

Workflow compiled successfully!


Test - Prompt Engineering

In [18]:
inputs = {"question": "What is prompt engineering?"}

for output in app.stream(inputs):
    for key, value in output.items():
        if "generation" in value:
            print(f"\nAnswer: {value['generation']}")

---ROUTE QUESTION---
---ROUTE: VECTORSTORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE---
---GRADE: RELEVANT---
---GRADE: RELEVANT---
---ASSESS DOCUMENTS---
---DECISION: GENERATE---
---GENERATE---
---CHECK HALLUCINATIONS---
---GROUNDED IN DOCUMENTS---
---ADDRESSES QUESTION---

Answer: Prompt engineering refers to methods for communicating with language models to achieve desired outcomes without updating the model weights. It is an empirical science that requires experimentation and heuristics. The goal of prompt engineering is to achieve alignment and model steerability.


Test - Current Events

In [19]:
inputs = {"question": "Who are the Bears expected to draft first in the NFL draft?"}

for output in app.stream(inputs):
    for key, value in output.items():
        if "generation" in value:
            print(f"\nAnswer: {value['generation']}")

---ROUTE QUESTION---
---ROUTE: WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---GROUNDED IN DOCUMENTS---
---ADDRESSES QUESTION---

Answer: The Chicago Bears are expected to draft Caleb Banks, a defensive lineman from the University of Florida, with their first-round pick at number 25 overall. Banks is a 6-foot-6, 330-pound prospect with high upside and the potential to bring interior disruption to the Bears' defense. He has length and power, and his rare size is hard to find.


Test - Agent Memory

In [20]:
inputs = {"question": "What are the types of agent memory?"}

for output in app.stream(inputs):
    for key, value in output.items():
        if "generation" in value:
            print(f"\nAnswer: {value['generation']}")

---ROUTE QUESTION---
---ROUTE: VECTORSTORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE---
---GRADE: RELEVANT---
---GRADE: NOT RELEVANT---
---ASSESS DOCUMENTS---
---DECISION: WEB SEARCH---
---WEB SEARCH---
---GENERATE---
---CHECK HALLUCINATIONS---
---GROUNDED IN DOCUMENTS---
---ADDRESSES QUESTION---

Answer: There are several types of agent memory, including:

1. Episodic Memory: stores specific events and interactions, such as conversational history and summaries of key events.
2. Semantic Memory: stores facts and concepts, providing the agent's organized knowledge repository.
3. Procedural Memory: stores workflows and skills, enabling agents to execute complex multi-step processes automatically.
4. Associative Memory: creates and maintains relationships between different pieces of information, allowing agents to make inferences.
5. Factual Memory: stores user-specific and environment-specific information, including preferences, history, and ongoing objectives.
6. Short-term Memory: 